<a href="https://colab.research.google.com/github/megha-puthukudi/main-project/blob/ResNet50V2/_content_drive_MyDrive_colabtest_vecicle_damage_ReNet50v2_epoch_60_new_dataset_splited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix
import seaborn as sns  # Added seaborn for visualization

# Define your data directories
train_dir = '/content/drive/MyDrive/new dataset/train'
val_dir = '/content/drive/MyDrive/new dataset/validation'
test_dir = '/content/drive/MyDrive/new dataset/test'

In [3]:
# Define image size and batch size
img_size = (224, 224)
batch_size = 16

In [4]:
# Create data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 600 images belonging to 2 classes.
Found 90 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [5]:
# Load the pre-trained ResNet50V2 model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a dense layer for classification
x = GlobalAveragePooling2D()(base_model.output)
predictions = Dense(2, activation='softmax')(x)  # Assuming 2 classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


94668760/94668760 [==============================] - 1s 0us/step


In [6]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [7]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=60,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size
)

Epoch 1/60
37/37 [==============================] - 164s 4s/step - loss: 0.8992 - accuracy: 0.5017 - val_loss: 0.6604 - val_accuracy: 0.6250
Epoch 2/60
37/37 [==============================] - 11s 290ms/step - loss: 0.6484 - accuracy: 0.6267 - val_loss: 0.5563 - val_accuracy: 0.7125
Epoch 3/60
37/37 [==============================] - 11s 290ms/step - loss: 0.5167 - accuracy: 0.7397 - val_loss: 0.4335 - val_accuracy: 0.8000
Epoch 4/60
37/37 [==============================] - 10s 263ms/step - loss: 0.4642 - accuracy: 0.7894 - val_loss: 0.4109 - val_accuracy: 0.8375
Epoch 5/60
37/37 [==============================] - 10s 270ms/step - loss: 0.3896 - accuracy: 0.8425 - val_loss: 0.3780 - val_accuracy: 0.8625
Epoch 6/60
37/37 [==============================] - 11s 298ms/step - loss: 0.3520 - accuracy: 0.8596 - val_loss: 0.3435 - val_accuracy: 0.8875
Epoch 7/60
37/37 [==============================] - 10s 283ms/step - loss: 0.3444 - accuracy: 0.8596 - val_loss: 0.3423 - val_accuracy: 0.8750
E